In [22]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
#from plotly.offline import init_notebook_mode, iplot
#import plotly.graph_objs as go
import os
import warnings

# 파일 불러와서 열이름 바꾸기
korean, long_term_frgn, short_term_frgn, resident, card_cnt, card_amt, waste_cnt
(이거 예시는 card_amt임)

In [374]:
##### 파일 불러오기 #####
df = pd.read_csv("C:\\Users\\asus\\BigContest\\files\\data_preprocessing\\1_food_waste_preprocessing.csv", encoding='cp949')

# 시계열 피처단위로 변경
df['base_date'] = pd.to_datetime(df['base_date'])

# base_date를 index로 설정
df.index = df['base_date']
df.columns = ['base_date', 'emd_cd', 'waste_cnt', 'waste_amt']  ##### 각자 데이터에 맞게 열이름 변경하기 #####
df

,base_date,emd_cd,waste_cnt,waste_amt
base_date,,,,
2019-11-30,2019-11-30,50110250,620,2279550
2019-12-31,2019-12-31,50110250,8178,31493650
2020-01-31,2020-01-31,50110250,13042,49632850
2020-02-29,2020-02-29,50110250,12264,46197050
2020-03-31,2020-03-31,50110250,14316,52561750
...,...,...,...,...
2021-02-28,2021-02-28,50130620,4563,15566250
2021-03-31,2021-03-31,50130620,4920,17300600
2021-04-30,2021-04-30,50130620,5256,17646850


In [375]:
# 필요 없는 열 제거
data = df.drop(['base_date', 'waste_amt'], 1)     ##### 각자 필요없는 열 제거 #####
data

,emd_cd,waste_cnt
base_date,,
2019-11-30,50110250,620
2019-12-31,50110250,8178
2020-01-31,50110250,13042
2020-02-29,50110250,12264
2020-03-31,50110250,14316
...,...,...
2021-02-28,50130620,4563
2021-03-31,50130620,4920
2021-04-30,50130620,5256


In [376]:
# 한 행정동의 행만 추출
sub_area = data['emd_cd'] == 50130580  ##### 각 행정동코드 입력 #####

# base_date, card_amt 만 갖는 df 생성
sub_df = data[sub_area]
sub_df = sub_df.drop('emd_cd', 1)
sub_df

,waste_cnt
base_date,
2018-01-31,15597
2018-02-28,15013
2018-03-31,16243
2018-04-30,15485
2018-05-31,16979
2018-06-30,17531
2018-07-31,21064
2018-08-31,21462
2018-09-30,20800


In [377]:
# series 형으로 변환
sub_df.to_dict('series')

{'waste_cnt': base_date
 2018-01-31    15597
 2018-02-28    15013
 2018-03-31    16243
 2018-04-30    15485
 2018-05-31    16979
 2018-06-30    17531
 2018-07-31    21064
 2018-08-31    21462
 2018-09-30    20800
 2018-10-31    19207
 2018-11-30    17721
 2018-12-31    17939
 2019-01-31    14060
 2019-02-28     8893
 2019-03-31     5778
 2019-04-30     5789
 2019-05-31    15830
 2019-06-30    18250
 2019-07-31    21594
 2019-08-31    22292
 2019-09-30    20694
 2019-10-31    19001
 2019-11-30    17979
 2019-12-31    17069
 2020-01-31    15526
 2020-02-29    14629
 2020-03-31    16255
 2020-04-30    15418
 2020-05-31    16819
 2020-06-30    17108
 2020-07-31    18648
 2020-08-31    25071
 2020-09-30    22303
 2020-10-31    19742
 2020-11-30    15516
 2020-12-31    15749
 2021-01-31    16533
 2021-02-28    16133
 2021-03-31    17363
 2021-04-30    16810
 2021-05-31    16953
 2021-06-30    14277
 Name: waste_cnt, dtype: int64}

# ARIMA

In [378]:
# ARIMA에 적합한 float로 바꾸는 과정
sub_df_float = sub_df[:].astype(np.float)
#sub_df_float.tail()

In [379]:
# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
from statsmodels.tsa.stattools import adfuller

result = adfuller(sub_df)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))
    
##### 여기서 p-value 확인 -> 0.05보다 크면 아래로 내려가서 차분하기 / 작으면 차분 생략(아래 셀 생략) #####

ADF Statistic : -4.113967
p-value : 0.000918
Critical Values :
	1%: -3.610400
	5%: -2.939109
	10%: -2.608063


In [380]:
##### 이건 설치하는 거라 처음 돌릴 때 한 번만 주석 풀어서 설치하고 그 이후로는 필요없음! #####

#!pip install pmdarima

In [381]:
from pmdarima.arima import auto_arima

model_arima= auto_arima(sub_df,trace=True, error_action='ignore',suppress_warnings=True,stepwise=False,seasonal=True)

model_arima.fit(sub_df)

##### 여기서 아래쪽에 'Best model:  ARIMA(0,1,0)(0,0,0)[1] intercept' 이렇게 나오면 (0,1,0) 가져다 쓸거임 #####
##### ARIMA(0,1,0) 여기 숫자 세 개만 잘 보면 됨 !!!! #####

 ARIMA(0,0,0)(0,0,0)[1] intercept   : AIC=814.393, Time=0.01 sec
 ARIMA(0,0,1)(0,0,0)[1] intercept   : AIC=788.675, Time=0.05 sec
 ARIMA(0,0,2)(0,0,0)[1] intercept   : AIC=776.569, Time=0.09 sec
 ARIMA(0,0,3)(0,0,0)[1] intercept   : AIC=772.777, Time=0.07 sec
 ARIMA(0,0,4)(0,0,0)[1] intercept   : AIC=777.005, Time=0.04 sec
 ARIMA(0,0,5)(0,0,0)[1] intercept   : AIC=776.567, Time=0.13 sec
 ARIMA(1,0,0)(0,0,0)[1] intercept   : AIC=782.666, Time=0.01 sec
 ARIMA(1,0,1)(0,0,0)[1] intercept   : AIC=779.799, Time=0.02 sec
 ARIMA(1,0,2)(0,0,0)[1] intercept   : AIC=774.682, Time=0.03 sec
 ARIMA(1,0,3)(0,0,0)[1] intercept   : AIC=776.162, Time=0.04 sec
 ARIMA(1,0,4)(0,0,0)[1] intercept   : AIC=inf, Time=0.21 sec
 ARIMA(2,0,0)(0,0,0)[1] intercept   : AIC=774.438, Time=0.02 sec
 ARIMA(2,0,1)(0,0,0)[1] intercept   : AIC=777.304, Time=0.07 sec
 ARIMA(2,0,2)(0,0,0)[1] intercept   : AIC=775.513, Time=0.05 sec
 ARIMA(2,0,3)(0,0,0)[1] intercept   : AIC=777.584, Time=0.07 sec
 ARIMA(3,0,0)(0,0,0)[1] inter

ARIMA(order=(3, 0, 0), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True)

In [382]:
# ARIMA 모델 
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델을 학습합니다.
model = ARIMA(sub_df.waste_cnt.values, order=(3,0,0))     ##### 여기 order=(0,1,0)에 위에서 나온 숫자 넣는거 #####
                                    ##### 그리고 sub_df.card_amt.values에서 가운데 card_amt 각자걸로 바꾸기 #####

#trend : constant를 가지고 있는지, c - constant / nc - no constant
#disp : 수렴 정보를 나타냄
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

C:\Users\asus\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


                              ARMA Model Results                              
Dep. Variable:                      y   No. Observations:                   42
Model:                     ARMA(3, 0)   Log Likelihood                -381.193
Method:                       css-mle   S.D. of innovations           2077.498
Date:                Fri, 10 Sep 2021   AIC                            772.385
Time:                        15:43:45   BIC                            781.073
Sample:                             0   HQIC                           775.570
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.693e+04    677.647     24.981      0.000    1.56e+04    1.83e+04
ar.L1.y        0.9418      0.146      6.435      0.000       0.655       1.229
ar.L2.y       -0.1215      0.208     -0.584      0.5

# 예측하기

In [383]:
# 2단위 이후의 예측결과
fore = model_fit.forecast(steps=2)
print(fore)

##### 맨 앞에 나온 'array([10681271.81014634, 10741037.02029268])' 이게 차례대로 7, 8월 예측값임 #####
##### 7월 : 10681271.81014634, 8월 : 10741037.02029268 #####
##### 이 값 가져다 엑셀파일에 입력하기 #####

(array([14463.76732761, 14922.13900936]), array([2077.49800853, 2853.82375885]), array([[10391.94605294, 18535.58860227],
       [ 9328.74722379, 20515.53079494]]))
